**data prep for dimensionality reduction**

In [ ]:
import os
import glob
from tqdm import tqdm
import pickle
import mne
from pyts.approximation import SymbolicAggregateApproximation # https://pyts.readthedocs.io/en/stable/auto_examples/approximation/plot_sax.html
mne.set_log_level('WARNING')

N_BINS = 8
SAMPLING_RATE = 128

DATA_CHANNEL = 'EEG FZ-REF'
REFERENCE_CHANNEL = 'EEG CZ-REF'

# TRANSFORMER INPUT CONFIG
ALPHABET_LEN = .03 #sec : 30 milliseconds
WORD_LEN = 10 #sec : 10 seconds
WORDS_PER_SENT = 10

abnorm_datapath = r"/mnt/ssd_4tb_0/shared/TUH_EEG_ABNORMAL/v2.0.0/edf/eval/abnormal"
norm_datapath = r"/mnt/ssd_4tb_0/shared/TUH_EEG_ABNORMAL/v2.0.0/edf/eval/normal"

abnorm_edf_files = glob.glob(f'{abnorm_datapath}/**/*.edf', recursive=True)
norm_edf_files = glob.glob(f'{norm_datapath}/**/*.edf', recursive=True)

In [ ]:
DATA_PATH = r"/mnt/ssd_4tb_0/shared/TUH_EEG_ABNORMAL/"
edf_files = glob.glob(f'{DATA_PATH}/**/*.edf', recursive=True)
TEST_FILES = edf_files[::7]
TRAIN_FILES = list(filter(lambda x: x not in TEST_FILES, edf_files))
TO_TRANSFORM_FILES = TRAIN_FILES[::10]

ALL_FILES = edf_files

In [ ]:
TEST_LABELS = []
for files in TEST_FILES:
    t = files.split('/')
    label = t[8]
    TEST_LABELS.append(label)

In [ ]:
len(TEST_FILES), len(TEST_LABELS)

In [ ]:
import os
import pickle

folder_path = '/home/john_zhang/EEG_NLP/llm_for_eeg/eval'  # Replace with the path to your folder
file_extension = '.pkl'

# Get a list of all .pkl files in the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith(file_extension)]

# Create an empty list to store the data from each file
data_array = []

# Iterate over the files and load the data into the array
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        data_array.append(data[0])


In [ ]:
len(data_array),len(TEST_FILES),len(TEST_LABELS)

In [ ]:
#sample n from normal/abnormal
sample_size = 50

import numpy as np
# arr1 = np.array(patient_id)
arr2 = np.array(TEST_LABELS)
# Find indices of sequences with label A and B
indices_A = np.where(arr2 == 'abnormal')[0]
indices_B = np.where(arr2 == 'normal')[0]


# Randomly shuffle the indices
np.random.shuffle(indices_A)
np.random.shuffle(indices_B)

# print(indices_A[:100])
# print(indices_B)
# print(np.intersect1d(indices_A,indices_B))

abnorm_samples = indices_A[:sample_size]
norm_samples = indices_B[:sample_size]

sample_labels = ['abnormal']*len(abnorm_samples) + ['normal']*len(norm_samples)


# Select the first 100 sequences with label A and B
# abnorm = [train_data[x] for x in indices_A[:nums]]
# abnorm_patient_id = arr1[indices_A[:nums]]

# norm = [train_data[x] for x in indices_B[:nums]]
# norm_patient_id = arr1[indices_B[:nums]]


In [ ]:

s = 1000
TOKEN_LENGTH = 30 #ms
WORD_LENGTH = 10*s #s
SENTENCE_LENGTH = 100*s #s

TOKENIZER_BATCH_SIZE = 256


In [ ]:
train_data = []
labels = []
patient_id = []
for i in abnorm_samples:
    cur_sbj = data_array[i]
    wrd_step = int(WORD_LENGTH/TOKEN_LENGTH)
    words = ["".join(cur_sbj[i:i+wrd_step]) for i in range(0,len(cur_sbj),wrd_step)]
    
    snt_step = int(SENTENCE_LENGTH/WORD_LENGTH)
    #print(words[0])
    sentences = [" ".join(words[i:i+snt_step]) for i in range(0,len(words),snt_step)]
    labels += len(sentences)*[TEST_LABELS[i]]
    
    patient_id.append(len(sentences))
   
    
    train_data.append(sentences)
for i in norm_samples:
    cur_sbj = data_array[i]
    wrd_step = int(WORD_LENGTH/TOKEN_LENGTH)
    words = ["".join(cur_sbj[i:i+wrd_step]) for i in range(0,len(cur_sbj),wrd_step)]
    
    snt_step = int(SENTENCE_LENGTH/WORD_LENGTH)
    #print(words[0])
    
    sentences = [" ".join(words[i:i+snt_step]) for i in range(0,len(words),snt_step)]
    labels += len(sentences)*[TEST_LABELS[i]]
    patient_id.append(len(sentences))
   
    
    train_data.append(sentences)

In [ ]:
labels.count('normal'),labels.count('abnormal'),len(patient_id), len(train_data), len(sentences)

In [ ]:
def write_2d_array_to_txt(array, filename):
    with open(filename, 'w') as file:
        for row in array:
            file.write(''.join(row) + '\n')

tmp_arr = []
for sent in train_data:
    tmp_arr += sent

# Specify the filename
output_file = 'train_data3.txt'

# Write the array to the .txt file
write_2d_array_to_txt(tmp_arr, output_file)

**Running test data through the model**

In [34]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json
import numpy as np

/home/john_zhang/.conda/envs/eeg_nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-20 13:19:21.681108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 13:19:22.939667: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/cuda/lib:/usr/local/cuda/lib64:
2023-06-20 13:19:22.940195: W tensorflow/compiler/xla/stream_executor/pla

In [35]:
# load the model checkpoint
model_path = "/home/john_zhang/EEG_NLP/llm_for_eeg/pretrained-bert"
model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-7000"))


loading configuration file /home/john_zhang/EEG_NLP/llm_for_eeg/pretrained-bert/checkpoint-7000/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.28.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/john_zhang/EEG_NLP/llm_for_eeg/pretrained-bert/checkpoint-7000/pytorch_model.bin
Generate config GenerationConfig {
  "_from_model_config": true,
  "pad_token_id": 0,
  "transformers_version": "4.28.0.dev0"
}

All model checkpoint weights were us

In [36]:
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file('bpev2.tokenizer.json')
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
tokenizer = fast_tokenizer
tokenizer.pad_token = "[PAD]"
tokenizer.mask_token = "[MASK]"


In [37]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["output.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = True

In [38]:
dataset = load_dataset("text", data_files=["train_data3.txt"])

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1942.71it/s]


Dataset text downloaded and prepared to /home/john_zhang/.cache/huggingface/datasets/text/default-2b0679017126b8ad/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 805.98it/s]


In [39]:
len(dataset['train']['text'][0])

3339

In [40]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length",
                   max_length=max_length, return_special_tokens_mask=True)
def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation
# tokenizing the train dataset
test_dataset = dataset.map(encode, batched=True)

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as PyTorch tensors
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

In [41]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 5168
    })
})

In [42]:
len(test_dataset['train'])

5168

In [43]:
input_ids = test_dataset['train']['input_ids'][0:47]  # Add a dimension for batch_size
print(input_ids)
attention_mask = test_dataset['train']['attention_mask'][0:47]  # Add a dimension for batch_size
print(attention_mask)
output = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)

tensor([[ 157, 5845,  157,  ...,    3,    3,    3],
        [   6, 1364,  303,  ...,  765,  583,    3],
        [   8,  351,   85,  ...,    3,    3,    3],
        ...,
        [  38,  296,  482,  ...,    3,    3,    3],
        [  14, 7257, 1558,  ...,    3,    3,    3],
        [5591,   57, 2164,  ...,    3,    3,    3]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


**smaller samples**

In [48]:
from tqdm import tqdm

start = 0
avg_output = []
labels_small = []

for i in tqdm(range(len(train_data))):
    end = start + patient_id[i]
    #print(start,end)
    input_ids = test_dataset['train']['input_ids'][start:end]  
    attention_mask = test_dataset['train']['attention_mask'][start:end]
    start += patient_id[i]
    output = model(input_ids, attention_mask=attention_mask, output_hidden_states=True) 
    #print(output.hidden_states[-1].shape)
    tmp_avg1 = np.mean(output.hidden_states[-1].detach().numpy(),axis=1)
    #print(tmp_avg1.shape)
    tmp_avg2 = np.mean(tmp_avg1,axis = 0)
    #print(tmp_avg2.shape)
    avg_output.append(tmp_avg2)
    labels_small.append(sample_labels[i])
    
    

100%|██████████| 100/100 [15:41<00:00,  9.42s/it]


In [49]:
len(avg_output),len(labels_small)

(100, 100)

In [25]:
# # averaging individual segments, then average into (batch_size, hidden_output_size)
# from tqdm import tqdm

# start = 0
# avg_output = []
# labels_small = []
# for i in tqdm(range(len(abnorm))):
#     end = start + abnorm_patient_id[i]
#     #print(start,end)
#     input_ids = test_dataset['train']['input_ids'][start:end]  
#     attention_mask = test_dataset['train']['attention_mask'][start:end]
#     start += abnorm_patient_id[i]
#     output = model(input_ids, attention_mask=attention_mask, output_hidden_states=True) 
#     #print(output.hidden_states[-1].shape)
#     tmp_avg1 = np.mean(output.hidden_states[-1].detach().numpy(),axis=1)
#     #print(tmp_avg1.shape)
#     tmp_avg2 = np.mean(tmp_avg1,axis = 0)
#     #print(tmp_avg2.shape)
#     avg_output.append(tmp_avg2)
#     labels_small.append('abnormal')

# # for i in tqdm(range(len(norm))):
# #     end = start + norm_patient_id[i]
# #     #print(start,end)
# #     input_ids = test_dataset['train']['input_ids'][start:end]  
# #     attention_mask = test_dataset['train']['attention_mask'][start:end]
# #     start += abnorm_patient_id[i]
# #     output = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
# #     #print(output.hidden_states[-1].shape)
# #     tmp_avg1 = np.mean(output.hidden_states[-1].detach().numpy(),axis=1)
# #     #print(tmp_avg1.shape)
# #     tmp_avg2 = np.mean(tmp_avg1,axis = 0)
# #     #print(tmp_avg2.shape)
# #     avg_output.append(tmp_avg2)
# #     labels_small.append('normal')
    
# print(start)

100%|██████████| 50/50 [07:04<00:00,  8.49s/it]

2473


In [26]:
import pickle
with open('embeddings/cluster_abnorm_data_v4.pkl', 'wb') as f:
    pickle.dump(avg_output, f)
with open('embeddings/cluster_abnorm_label_v4.pkl', 'wb') as f:
    pickle.dump(labels_small, f)

In [50]:
import pickle
with open('embeddings/cluster_data_v3.pkl', 'wb') as f:
    pickle.dump(avg_output, f)
with open('embeddings/cluster_label_v3.pkl', 'wb') as f:
    pickle.dump(labels_small, f)

In [18]:
# averaging individual segments, then average into (batch_size, hidden_output_size)

start = 0
avg_output = []
for i in range(len(patient_id)):
    end = start + patient_id[i]
    #print(start,end)
    input_ids = test_dataset['train']['input_ids'][start:end]  
    attention_mask = test_dataset['train']['attention_mask'][start:end]
    start += patient_id[i]
    output = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
    #print(output.hidden_states[-1].shape)
    tmp_avg1 = np.mean(output.hidden_states[-1].detach().numpy(),axis=1)
    #print(tmp_avg1.shape)
    tmp_avg2 = np.mean(tmp_avg1,axis = 0)
    #print(tmp_avg2.shape)
    avg_output.append(tmp_avg2)



: 

: 

In [19]:
# only averaging individual segments
avg_output = []
for i in range(len(labels)):
    #print(start,end)
    input_ids = test_dataset['train']['input_ids'][i].unsqueeze(0)
    attention_mask = test_dataset['train']['attention_mask'][i].unsqueeze(0)
    output = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
    # print(output.hidden_states[-1].shape)
    tmp_avg1 = np.mean(output.hidden_states[-1].detach().numpy(),axis=1)
    # print(tmp_avg1.shape)
    tmp_avg2 = np.mean(tmp_avg1,axis = 0)
    # print(tmp_avg2.shape)
    
    avg_output.append(tmp_avg2)
    # break

In [69]:
np.array(avg_output).shape

(428, 76)

In [ ]:
import pickle
with open('embeddings/cluster_data_v3.pkl', 'wb') as f:
    pickle.dump(avg_output, f)
with open('embeddings/cluster_label_v3.pkl', 'wb') as f:
    pickle.dump(TEST_LABELS, f)

In [20]:
import pickle
with open('embeddings/cluster_data2_v3.pkl', 'wb') as f:
    pickle.dump(avg_output, f)
with open('embeddings/cluster_label2_v3.pkl', 'wb') as f:
    pickle.dump(labels, f)

**SAX from scratch**

In [7]:
def single_signal(raw_data, signals):
    fz_idx = raw_data.ch_names.index(DATA_CHANNEL)
    cz_idx = raw_data.ch_names.index(REFERENCE_CHANNEL)
    res = signals[fz_idx] - signals[cz_idx]
    return res
all_single_signals = []

def process_edf(ef):
    raw_data = mne.io.read_raw_edf(ef, preload=True)
    raw_data = raw_data.resample(SAMPLING_RATE) 
    signals, times = raw_data[:]
    s_signal = single_signal(raw_data, signals)
    return s_signal


# for ef in tqdm(TRAIN_FILES):
#     raw_data = mne.io.read_raw_edf(ef, preload=True)
#     raw_data = raw_data.resample(SAMPLING_RATE) 
#     signals, times = raw_data[:]
#     s_signal = single_signal(raw_data, signals)
#     all_single_signals.append(s_signal)

In [ ]:
from tqdm.autonotebook import tqdm
import concurrent.futures

def _yield_args(all_file_paths):
    for x in all_file_paths:
        yield x
        
all_single_signals = []
with tqdm(total=len(TRAIN_FILES)) as pbar, \
    concurrent.futures.ProcessPoolExecutor(max_workers=24) as executor:
    for result in executor.map(process_edf, _yield_args(TRAIN_FILES)):
         all_single_signals.append(result)
         pbar.update(1)









In [9]:
import numpy as np

def pad_jagged_array(jagged_array, padding_value=-1):
    # Find the maximum length of any row
    max_length = max(len(row) for row in jagged_array)

    # Initialize the padded array
    padded_array = np.full((len(jagged_array), max_length), padding_value)

    # Copy the values from the jagged array into the padded array
    for i, row in enumerate(jagged_array):
        padded_array[i, :len(row)] = row

    return padded_array


padded_signals = pad_jagged_array(all_single_signals)

In [ ]:
import pickle
with open('sax_train_dataset.pkl', 'wb') as f:
    pickle.dump(all_single_signals, f)

In [8]:
sax = SymbolicAggregateApproximation(n_bins=N_BINS, strategy='quantile')

In [ ]:
flat_signals_binned = sax.fit_transform(padded_signals)


In [9]:
for ef in tqdm(TEST_FILES):
    raw_data = mne.io.read_raw_edf(ef, preload=True)
    raw_data = raw_data.resample(SAMPLING_RATE) 
    signals, times = raw_data[:]
    s_signal = single_signal(raw_data, signals)

    # SAX transformation
    X_sax = sax.transform([s_signal])

    
    payload = {
        #    "raw_signals": signals,
        #    "times": times,
        #    "quantized_signals": X_sax,
        }
    payload = X_sax
    with open(f"/home/john_zhang/EEG_NLP/llm_for_eeg/eval/{os.path.basename(ef)}.pkl", "wb") as df:
        pickle.dump(payload, df)

100%|██████████| 428/428 [06:52<00:00,  1.04it/s]


In [ ]:
for ef in tqdm(TRAIN_FILES):
    raw_data = mne.io.read_raw_edf(ef, preload=True)
    raw_data = raw_data.resample(SAMPLING_RATE) 
    signals, times = raw_data[:]
    s_signal = single_signal(raw_data, signals)

    # SAX transformation
    X_sax = sax.transform([s_signal])

    
    payload = {
        #    "raw_signals": signals,
        #    "times": times,
        #    "quantized_signals": X_sax,
        }
    payload = X_sax
    with open(f"/home/john_zhang/EEG_NLP/llm_for_eeg/train/{os.path.basename(ef)}.pkl", "wb") as df:
        pickle.dump(payload, df)